In [5]:
import numpy as np

In [6]:
def bernoulli(p, shape):
    return np.random.binomial(1, p, shape)

In [7]:
#general idea: if probability is set to None, then using an operator is random (we sample uniformly prob)
# but if we set prob to 0, then the operator is not used

In [253]:
class Recombination(object):
    def __init__(self, type='uniform', prob=None, num_cutting_points=0):
        assert type in ['k-points', 'uniform']
        self.type = type
        
        if prob is not None:
            assert (prob >= 0) and (prob <= 1), 'Probability must be in [0, 1]!'
        self.prob = prob  #what is the probability of applying recombination (the lower it is, the lower chance it's applied)
        
        if type in ['k-points']:
            assert num_cutting_points > 0, 'There must be at least 1 cutting point!'
        self.num_cutting_points = num_cutting_points
        
    def recombine(self, x):
        # pick which points to recombine
        if self.prob is None:
            b = bernoulli(np.random.rand(1), (x.shape[0],))
        else:
            b = bernoulli(self.prob, (x.shape[0],))
        
        if np.sum(b) > 0:
            x_remain = x[(1. - b).astype(bool)]

            # take first parent
            parent_1 = x[b.astype(bool)]
            # assign second parent (shuffle x for that)
            indices = np.arange(parent_1.shape[0])
            np.random.shuffle(indices)
            parent_2 = parent_1[indices,:]

            if self.type == 'uniform':
                p = bernoulli(0.5, (1, parent_1.shape[1]))

            elif self.type == 'k-points':
                if self.num_cutting_points >= parent_1.shape[1]:
                    self.num_cutting_points = parent_1.shape[1]-1
                
                cutting_points = np.random.permutation(np.arange(parent_1.shape[1]))[:self.num_cutting_points]
                cutting_points = np.sort(cutting_points)

                p = np.zeros((1, parent_1.shape[1]))
                if self.num_cutting_points == 1:
                    r = 1
                else:
                    r = 0    
                start = 0
                for j in cutting_points:
                    p[0, start:j] = r
                    r = np.mod(r+1,2)
                    start = j
                
            child = p * parent_1 + (1.-p) * parent_2

            # return indices of remaining points (in order to avoid re-evaluating fitness for them again!) and children
            return (1. - b).astype(bool), child
        else:
            return (1. - b).astype(bool), None

In [213]:
class Mutation(object):
    def __init__(self, type='binary', prob=None, std=None):
        if prob is not None:
            assert (prob >= 0) and (prob <= 1), 'Probability must be in [0, 1]!'
        assert type in ['binary', 'gaussian']
        if type == 'gaussian':
            assert (std is not None) and (std > 0.)
        
        self.prob = prob #probability of CHANGING a variable
        self.type = type
        self.std = std
    
    def mutate(self, x):
        x_shape = x.shape
        
        if self.prob == 0.:
            return x
        else:
            if self.prob is None:
                mask = bernoulli(np.random.rand(1), x_shape)
            else:
                mask = bernoulli(self.prob, x_shape)

            if self.type == 'binary':
                x = np.mod(x + mask, 2)
            elif self.type == 'gaussian':
                gauss = np.random.normal(0., self.std, x_shape)
                x = x + mask * gauss
            return x

In [358]:
class Selection(object):
    def __init__(self, type='proportional'):
        assert type in ['proportional'], 'Wrong type!'
        self.type = type
    
    def select(self, x, f, population_size=1):
        assert (population_size > 0) and (population_size < x.shape[0]), 'Too small or too large population size!'
        
        if self.type == 'proportional':
            exp_f = np.exp(f)
            probability = exp_f / np.sum(exp_f)
            
            indices = np.random.choice(x.shape[0], population_size, replace=False, p=probability)
            
        return x[indices], f[indices]

In [1]:
class EA(object):
    def __init__(self, fun, args, x_0, add_fun=None, recombination_type='uniform', recombination_prob=0.5, recombination_num_cutting_points=1, mutation_type='gaussian', mutatiob_prob=0.5, selection_type='proportional', population_size=100):
        self.recombination = Recombination(recombination_type, recombination_prob, recombination_num_cutting_points)
        self.mutation = Mutation(mutation_type, mutatiob_prob)
        self.selection = Selection(selection_type)
        
        self.population_size = population_size
        
        for i in x_shape:
            assert isinstance(i, int), 'Element: %r is not int!' % i
        self.x_shape = x_shape
        
        self.fun = fun
        self.add_fun = add_fun
        self.args = args
    
    def fitness(self, x):
        if add_fun is None:
            return self.fun(x, self.args)
        else:
            raise Error('Not implemented!')
    
    def recombination(self, x, f):
        if (self.recombination.prob is None) or (self.recombination.prob > 0.):
            indices, child = self.recombination.recombine(x)
            x_new = x[indices]
            f_new = f[indices]

            # if there are children, we need to evaluate them
            if child is not None:
                f_child = np.zeros((child.shape[0],))
                for i in range(child.shape[0]):
                    f_child[i] = self.fitness(child[i])

                x_new = np.concatenate((x_new, child), 0)
                f_new = np.concatenate((f_new, f_child), 0)

            return x_new, f_new
        else:
            return x, f
    
    def mutation(self, x, f):
        if (self.recombination.prob is None) or (self.recombination.prob > 0.): 
            x_new = self.mutation.mutate(x)
            #evaluate new fitness
            f_new
            return x_new, f_new
        else:
            return x, f
    
    def selection(self, x, f):
        x_new, f_new = selection(x, f, self.population_size)
        return x_new, f_new
    
    def step(self, x, f):
        if (self.recombination.prob is None) or (self.recombination.prob > 0):
            x, f = self.recombination(x, f)
        
        if (self.mutation.prob is None) or (self.mutation.prob > 0):
            x, f = self.mutation(x, f)
        
        x, f = self.selection(x, f)
    
    def run(self):
        x = np.random

In [317]:
ea = EA()
m = Mutation(prob=None, type='gaussian', std=0.1)

In [318]:
x = np.random.normal(2.1, 0.4, (4, 5))

In [319]:
x

array([[2.90585041, 1.92035371, 1.76917473, 2.53530911, 2.04091883],
       [2.79193349, 2.59436172, 2.79726666, 2.51279248, 2.75881388],
       [2.21005644, 2.04691787, 1.64914595, 2.23324892, 1.94610381],
       [2.7101758 , 2.14634473, 1.73876339, 1.6255571 , 2.89372391]])

In [320]:
r = Recombination(prob=0.7, type='uniform', num_cutting_points=2)

In [321]:
r.recombine(x)

(array([ True,  True,  True, False]),
 array([[2.7101758 , 2.14634473, 1.73876339, 1.6255571 , 2.89372391]]))

In [333]:
f = np.asarray([1.2, 0.1, 1.1, -0.14])

In [354]:
s = Selection()

In [355]:
x_new, f_new = s.select(x, f, 2)

[0.40007135 0.13317218 0.36199952 0.10475695]


In [356]:
x_new

array([[2.21005644, 2.04691787, 1.64914595, 2.23324892, 1.94610381],
       [2.79193349, 2.59436172, 2.79726666, 2.51279248, 2.75881388]])

In [357]:
f_new

array([1.1, 0.1])